In [ ]:
#| hide
from featsql.core import *

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_columns', None)

# featsql

> Create features with sql

This file will become your README and also the index of your documentation.

## Install

```sh
pip install featsql
```

## Configurando a engine

In [ ]:
url_db = "sqlite:///../.data/mydatabase.db" 
engine = create_engine(url_db)

## Visão inicial do público

Primeiro vamos observar o formato da tabela spine

In [ ]:
df_spine = pd.read_sql("SELECT * FROM tb_spine", engine)
df_spine.head()

,ID,SAFRA_REF,Target
0,4,2023-02-01,0
1,5,2023-02-01,0
2,6,2023-02-01,0
3,7,2023-02-01,0
4,10,2023-02-01,0


## Visão inicial da tabela de variáveis

A tabela de variáveis contém 3 variáveis, duas sendo numéricas e uma categórica. Existem mais ID's únicos e datas disponível nessa tabela do que na tabela spine.

In [ ]:
df_data = pd.read_sql("SELECT * FROM tb_feat", engine)
df_data.head()

,ID,SAFRA,FEAT_NUM1,FEAT_NUM2,FEAT_CAT1
0,1,2023-01-01,-20,-14,B
1,2,2023-01-01,34,14,B
2,3,2023-01-01,76,14,C
3,4,2023-01-01,15,44,C
4,5,2023-01-01,-81,56,B


## Criação de variáveis numéricas

In [ ]:
tb_publico='tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
target = 'Target'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]

In [ ]:
query_final = func_query_final(tb_publico, lista_janela,feat_num_lista, id, safra_ref, target, tb_feat, safra)

In [ ]:
df = pd.read_sql(query_final, engine)
df.head()

,ID,SAFRA_REF,Target,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AGV_1M,FEAT_NUM2_SUM_1M,FEAT_NUM2_MIN_1M,FEAT_NUM2_MAX_1M,FEAT_NUM2_AGV_1M,FEAT_NUM1_SUM_2M,FEAT_NUM1_MIN_2M,FEAT_NUM1_MAX_2M,FEAT_NUM1_AGV_2M,FEAT_NUM2_SUM_2M,FEAT_NUM2_MIN_2M,FEAT_NUM2_MAX_2M,FEAT_NUM2_AGV_2M,FEAT_NUM1_SUM_3M,FEAT_NUM1_MIN_3M,FEAT_NUM1_MAX_3M,FEAT_NUM1_AGV_3M,FEAT_NUM2_SUM_3M,FEAT_NUM2_MIN_3M,FEAT_NUM2_MAX_3M,FEAT_NUM2_AGV_3M
0,4,2023-02-01,0,15,15,15,15.0,44,44,44,44.0,15,15,15,15.0,44,44,44,44.0,15,15,15,15.0,44,44,44,44.0
1,5,2023-02-01,0,-81,-81,-81,-81.0,56,56,56,56.0,-81,-81,-81,-81.0,56,56,56,56.0,-81,-81,-81,-81.0,56,56,56,56.0
2,6,2023-02-01,0,-59,-59,-59,-59.0,23,23,23,23.0,-59,-59,-59,-59.0,23,23,23,23.0,-59,-59,-59,-59.0,23,23,23,23.0
3,7,2023-02-01,0,93,93,93,93.0,43,43,43,43.0,93,93,93,93.0,43,43,43,43.0,93,93,93,93.0,43,43,43,43.0
4,10,2023-02-01,0,92,92,92,92.0,20,20,20,20.0,92,92,92,92.0,20,20,20,20.0,92,92,92,92.0,20,20,20,20.0


In [ ]:
print(query_final)


    with 
    tb_public as (
        SELECT 
            *
        FROM tb_spine
    ),
    
        -- Criação de variáveis de janela de 1M
        tb_janela_1M as(
            SELECT 
                tb_public.ID,
                tb_public.SAFRA_REF,
                tb_public.Target,
                
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MAX_1M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_AGV_1M,
            
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MAX_1M,
            AVG(COALES